In [56]:
file_names = []
import os
for x in os.fwalk("marathon_training/"):
    for f in x[2]:
        file_names.append("marathon_training/"+f)
file_names.sort()

In [57]:
runs = []
import json
for fn in file_names:
    print("loading "+fn)
    with open(fn,"r",encoding="latin-1") as f:
        runs.append(json.load(f))

loading marathon_training/2019-03-17.Marathon_training_001.json
loading marathon_training/2019-03-19.Marathon_training_002.json
loading marathon_training/2019-03-26.Marathon_training_003.json
loading marathon_training/2019-03-29.Marathon_training_004.json
loading marathon_training/2019-03-31.Marathon_training_005.json
loading marathon_training/2019-04-08.Marathon_training_006.json
loading marathon_training/2019-04-10.Marathon_training_007.json
loading marathon_training/2019-04-12.Marathon_training_008.json
loading marathon_training/2019-04-14.Marathon_training_009.json
loading marathon_training/2019-04-16.Marathon_training_010.json
loading marathon_training/2019-04-18.Marathon_training_011.json
loading marathon_training/2019-04-20.Marathon_training_012.json
loading marathon_training/2019-04-22.Marathon_training_013.json
loading marathon_training/2019-04-24.Marathon_training_014.json
loading marathon_training/2019-04-26.Marathon_training_015.json
loading marathon_training/2019-04-28.Mar

In [3]:
from pprint import pprint
pprint(runs[0]['metadata'])

{'accessControlRuleDTO': {'typeId': 2, 'typeKey': 'private'},
 'activityId': 4022098483,
 'activityName': 'Marathon training 069',
 'activityTypeDTO': {'parentTypeId': 17,
                     'sortOrder': 3,
                     'typeId': 1,
                     'typeKey': 'running'},
 'activityUUID': {'uuid': 'b50fc19f-94bc-48ef-9ea3-90bb2dd5f0d2'},
 'description': 'short recovery run after breakfast, colder sunny morning, dry '
                'ground, pushing only for distance above 6km, harder start but '
                'ok 7/10',
 'eventTypeDTO': {'sortOrder': 7, 'typeId': 4, 'typeKey': 'training'},
 'isMultiSportParent': False,
 'locationName': 'Berlin',
 'metadataDTO': {'activityImages': [],
                 'agentApplicationInstallationId': None,
                 'agentString': None,
                 'associatedCourseId': None,
                 'associatedWorkoutId': None,
                 'autoCalcCalories': False,
                 'chartAvailability': {'showDistance': True,

In [20]:
from pprint import pprint
pprint(runs[0]["details"]["metricDescriptors"])

[{'key': 'directRunCadence',
  'metricsIndex': 0,
  'unit': {'factor': 1, 'id': 92, 'key': 'stepsPerMinute'}},
 {'key': 'sumElapsedDuration',
  'metricsIndex': 1,
  'unit': {'factor': 1000, 'id': 40, 'key': 'second'}},
 {'key': 'sumMovingDuration',
  'metricsIndex': 2,
  'unit': {'factor': 1000, 'id': 40, 'key': 'second'}},
 {'key': 'directLongitude',
  'metricsIndex': 3,
  'unit': {'factor': 1, 'id': 60, 'key': 'dd'}},
 {'key': 'sumDistance',
  'metricsIndex': 4,
  'unit': {'factor': 100, 'id': 1, 'key': 'meter'}},
 {'key': 'directTimestamp',
  'metricsIndex': 5,
  'unit': {'factor': 0, 'id': 120, 'key': 'gmt'}},
 {'key': 'sumDuration',
  'metricsIndex': 6,
  'unit': {'factor': 1000, 'id': 40, 'key': 'second'}},
 {'key': 'directCorrectedElevation',
  'metricsIndex': 7,
  'unit': {'factor': 100, 'id': 1, 'key': 'meter'}},
 {'key': 'directLatitude',
  'metricsIndex': 8,
  'unit': {'factor': 1, 'id': 60, 'key': 'dd'}},
 {'key': 'directSpeed',
  'metricsIndex': 9,
  'unit': {'factor': 0.1

In [15]:
pprint(runs[0]["details"]["activityDetailMetrics"][100]["metrics"][9]/0.1 * 1000 / 3600)

10.702777571148342


In [90]:
def get_run_metrics(run):
    run_metrics = {
        "sumDuration": {
            "index": None,
            "values": [],
            "factor": 1/60.,
            "unit": "time [min]"
        },
        "sumDistance": {
            "index": None,
            "values": [],
            "factor": 0.001,
            "unit": "distance [km]"
        },
        "directSpeed": {
            "index": None,
            "values": [],
            "factor": 10/3.6,
            "unit": "speed [km/h]"
        },
        "directVerticalSpeed": {
            "index": None,
            "values": [],
            "factor": 1,
            "unit": "m/s"
        },
        "directHeartRate": {
            "index": None,
            "values": [],
            "factor": 1,
            "unit": "heart rate [bpm]"
        }
    }
    for md in run["details"]["metricDescriptors"]:
        if md["key"] in run_metrics:
            run_metrics[md["key"]]["index"] = md["metricsIndex"]
    for am in run["details"]["activityDetailMetrics"]:
        for rm in run_metrics:
            if (run_metrics[rm]["index"] and am["metrics"][run_metrics[rm]["index"]]):
                run_metrics[rm]["values"].append( am["metrics"][run_metrics[rm]["index"]] )
            else:
                run_metrics[rm]["values"].append(None)
    return run_metrics


In [36]:
import matplotlib.pyplot
import numpy

In [111]:
def plot_run_metrics(figure, run_metrics, x, y, color):
    xraw = run_metrics[x]["values"]
    xvalues = numpy.multiply(
                numpy.array(xraw),
                run_metrics[x]["factor"],
                where=list(map(lambda v: v is not None, xraw))
              )
    yraw = run_metrics[y]["values"]
    yvalues = numpy.multiply(
                numpy.array(yraw),
                run_metrics[y]["factor"],
                where=list(map(lambda v: v is not None, yraw))
              )
    ymean = numpy.mean(yvalues[yvalues!=None])
    ystd = numpy.std(yvalues[yvalues!=None])
    yvalues = list(map(
        lambda v: v if v is None else (
            v if abs(v-ymean)<(1.65*ystd) else None
        ),
        yvalues
    ))
    graph = matplotlib.pyplot.plot( xvalues, yvalues, color=color )
    return graph


In [139]:
import dateutil
from datetime import datetime, timedelta


marathon_date = dateutil.parser.parse("2019-09-29")

In [146]:
def plot_runs_on_date(plot_date):
    figure = matplotlib.pyplot.figure()

    matplotlib.pyplot.title("{} days until marathon".format((marathon_date-plot_date).days))

    matplotlib.pyplot.xlabel("distance [km]")
    matplotlib.pyplot.ylabel("speed [km/h]")

    matplotlib.pyplot.xlim(0,45)
    matplotlib.pyplot.ylim(0,20)

    for run in runs:
        run_date = dateutil.parser.parse(run["metadata"]["summaryDTO"]["startTimeLocal"])
        if run_date <= plot_date:
            days_ago = (plot_date - run_date).days
            alpha = 0.05 if (days_ago>30) else 0.98-(0.03*days_ago)
            blue = 1 if (days_ago>30) else 0.03*days_ago
            red = 0 if (days_ago>30) else 1-(0.03*days_ago)
            color = (red, 0, blue, alpha) if (run_date < marathon_date) else (0,1,0,1)
            plot_run_metrics(figure, get_run_metrics(run), "sumDistance", "directSpeed", color)
    return figure

#plot_runs_on_date(dateutil.parser.parse("2019-06-13"))

In [148]:
for i in range(202):
    days_before = 200-i
    plot_date = marathon_date - timedelta(days=days_before)
    frame_path = "frames/{:0>3}.png".format(i)
    print("creating frame {}".format(frame_path))
    frame = plot_runs_on_date(plot_date)
    frame.savefig(frame_path)
    matplotlib.pyplot.close(frame)

creating frame frames/000.png
creating frame frames/001.png
creating frame frames/002.png
creating frame frames/003.png
creating frame frames/004.png
creating frame frames/005.png
creating frame frames/006.png
creating frame frames/007.png
creating frame frames/008.png
creating frame frames/009.png
creating frame frames/010.png
creating frame frames/011.png
creating frame frames/012.png
creating frame frames/013.png
creating frame frames/014.png
creating frame frames/015.png
creating frame frames/016.png
creating frame frames/017.png
creating frame frames/018.png
creating frame frames/019.png
creating frame frames/020.png
creating frame frames/021.png
creating frame frames/022.png
creating frame frames/023.png
creating frame frames/024.png
creating frame frames/025.png
creating frame frames/026.png
creating frame frames/027.png
creating frame frames/028.png
creating frame frames/029.png
creating frame frames/030.png
creating frame frames/031.png
creating frame frames/032.png
creating f